In [1]:
from src.query_tools.bigquery_connector import BigQuerySQLExecutor

In [2]:
bq = BigQuerySQLExecutor(project_id="wiq-gen-ai-rd-dev")

In [3]:
bq.test_connection()

True

In [4]:
bq.list_tables("finance")

['dim_date', 'fact_profit']

In [5]:
bq.execute_query("SELECT 1 as test_value")

QueryResult(rows=[{'test_value': 1}], column_names=['test_value'], row_count=1, execution_time_seconds=0.5142960548400879, query='SELECT 1 as test_value', is_truncated=False, total_rows_available=1, metadata={'job_id': 'dec17bbe-d30d-49b7-9cb9-ff73750fc29d', 'bytes_processed': 0, 'bytes_billed': 0, 'slot_millis': None, 'project_id': 'wiq-gen-ai-rd-dev', 'cache_hit': True})

In [6]:
from src.query_tools.query_functions import (
    weekly_store_performance_metrics,
    sales_vs_budget_by_fiscal_period,
    sales_vs_budget_by_fiscal_period,
    weekly_sales_and_gross_profit_by_subcategory,
    example_usage
)


In [7]:
bq.execute_query("SELECT 1 as test_value")

QueryResult(rows=[{'test_value': 1}], column_names=['test_value'], row_count=1, execution_time_seconds=0.4461081027984619, query='SELECT 1 as test_value', is_truncated=False, total_rows_available=1, metadata={'job_id': '2647a42d-2f70-4090-b510-05289f915059', 'bytes_processed': 0, 'bytes_billed': 0, 'slot_millis': None, 'project_id': 'wiq-gen-ai-rd-dev', 'cache_hit': True})

In [9]:
qry = example_usage()

2025-06-18 17:24:25.967 | DEBUG    | src.query_tools.query_functions:weekly_store_performance_metrics:96 - Generating weekly store performance query for salesorg: 1005, regions: ['1000SR01-SOPS', '1000SR02-SOPS']
2025-06-18 17:24:25.968 | INFO     | src.query_tools.query_functions:example_usage:514 - Generated weekly store performance query with parameters
2025-06-18 17:24:25.968 | DEBUG    | src.query_tools.query_functions:sales_vs_budget_by_fiscal_period:196 - Generating sales vs budget query for salesorg: 1005, timehierarchy: Last Completed Period TY
2025-06-18 17:24:25.969 | INFO     | src.query_tools.query_functions:example_usage:520 - Generated sales vs budget query with parameters


In [12]:

# q = "SELECT @p + @t as test_value, @p - @t as test_value_diff, @p * @t as test_value_product, @s"
# p = {"p": 40, "t": 2, "s": "thsi is a test"}
r = bq.execute_query(query=qry["weekly_performance"][0], parameters=qry["weekly_performance"][1])

In [10]:
r = bq.execute_query(query=qry["sales_vs_budget"][0], parameters=qry["sales_vs_budget"][1])

In [11]:
r.rows

[{'fiscal_period': '2025011',
  'actuals_sales': 3713298257.508712,
  'ops_budget': 3689890795.2314324,
  'merch_budget': 3689890970.4960036}]

In [11]:
params = {'salesorg': '1005',
 'regions': ['1000SR01-SOPS'],
 'timezone_1': 'Australia/Sydney',
 'timezone_2': 'Australia/Sydney',
 'timezone_3': 'Australia/Sydney',
 'timezone_4': 'Australia/Sydney',
 'comparison_days_1': 92,
 'comparison_days_2': 1,
 'comparison_days_3': 456,
 'comparison_days_4': 364,
 'comparison_days_offset_1': -1,
 'comparison_days_offset_2': -1}

In [ ]:
bq.execute_query(sql)

"\n    SELECT\n      fact_profit.fiscal_week,\n      CASE\n        WHEN fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 92 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 1 DAY), WEEK)\n        THEN 'TY'\n        WHEN fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 456 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 364 DAY), WEEK)\n        THEN 'LY'\n      END AS fiscal_year,\n      fact_profit.site,\n      fact_profit.site_description,\n      CASE\n        WHEN fact_profit.coles_competitor_distance = '<1 KM'\n        THEN 'Less than 1km'\n        WHEN fact_profit.coles_competitor_distance = '1-3 KM'\n        THEN '1km to 3km'\n        WHEN fact_profit.coles_competitor_distance = '3-5 KM'\n        THEN '3km to 5km'\n        WHEN fact_profit.coles_competitor_distance = '>5 KM'\

In [12]:
sql = """
    SELECT
      fact_profit.fiscal_week,
      CASE
        WHEN fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 92 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 1 DAY), WEEK)
        THEN 'TY'
        WHEN fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 456 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 364 DAY), WEEK)
        THEN 'LY'
      END AS fiscal_year,
      fact_profit.site,
      fact_profit.site_description,
      CASE
        WHEN fact_profit.coles_competitor_distance = '<1 KM'
        THEN 'Less than 1km'
        WHEN fact_profit.coles_competitor_distance = '1-3 KM'
        THEN '1km to 3km'
        WHEN fact_profit.coles_competitor_distance = '3-5 KM'
        THEN '3km to 5km'
        WHEN fact_profit.coles_competitor_distance = '>5 KM'
        THEN 'Greater than 5km'
        WHEN fact_profit.coles_competitor_distance = 'Same Centre'
        THEN 'Same Centre'
      END AS coles_competitor_distance,
      fact_profit.store_segment,
      SUM(fact_profit.sales_excltax) AS sales,
      -SUM(fact_profit.total_stock_adjustments) + SUM(fact_profit.dumps_cost) AS stock_adj,
      SUM(fact_profit.total_stock_loss) AS tsl,
      Safe_Divide(
        -SUM(fact_profit.total_stock_adjustments) + SUM(fact_profit.dumps_cost),
        SUM(fact_profit.sales_excltax)
      ) AS stk_adj_rate,
      SUM(fact_profit.items_sold) AS itemssold,
      SUM(fact_profit.transaction_cnt_site) AS customers
    FROM gcp-wow-ent-im-tbl-prod.gs_allgrp_fin_data.fin_group_profit_v AS fact_profit
    WHERE
      fact_profit.salesorg = @salesorg
      AND fact_profit.region IN UNNEST(@regions)
      AND NOT fact_profit.zone LIKE '%CLSD'
      AND NOT fact_profit.zone LIKE '%REGX'
      AND (
        fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 92 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 1 DAY), WEEK)
        OR fact_profit.calendar_day BETWEEN DATE_SUB(DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 456 DAY), WEEK), INTERVAL -1 DAY) AND DATE_TRUNC(DATE_SUB(CURRENT_DATE('Australia/Sydney'), INTERVAL 364 DAY), WEEK)
      )
    GROUP BY ALL
    ORDER BY
      fiscal_week;
    """


In [23]:
# # Create executor
# from src.query_tools.query_runner import create_retail_analytics_executor
# executor = create_retail_analytics_executor(
#     project_id="your-project-id",
#     credentials_path="/path/to/credentials.json"
# )

# # Test connection
# if executor.test_connection():
#     # Execute query
#     result = executor.weekly_store_performance_metrics(
#         salesorg="1005",
#         regions=["NSW", "VIC", "QLD"],
#         timezone_1="Australia/Sydney",
#         timezone_2="Australia/Sydney", 
#         timezone_3="Australia/Sydney",
#         timezone_4="Australia/Sydney",
#         comparison_days_1=92,
#         comparison_days_2=1,
#         comparison_days_3=456,
#         comparison_days_4=364,
#         comparison_days_offset_1=-1,
#         comparison_days_offset_2=-1
#     )
    
#     print(f"Query returned {result.row_count} rows")
#     print(f"Execution time: {result.execution_time_seconds:.2f}s")